In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


# Importing Libraries

In [2]:
from plotly import graph_objs as go
import plotly.express as px
import re
import string
from nltk.corpus import stopwords

In [3]:
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
primary_bgcolor = "#f4f0ea"

primary_green = px.colors.qualitative.Plotly[2]

# Importing Data

In [4]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding="latin-1")

df = df.dropna(how="any", axis=1)
df.columns = ['target', 'message']

df.head()

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.describe()

,target,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


# EDA

In [6]:
df['words_count'] = df['message'].apply(lambda x: len(x.split(' ')))
df['message_len'] = df['message'].apply(len)
df.head()

,target,message,words_count,message_len
0,ham,"Go until jurong point, crazy.. Available only ...",20,111
1,ham,Ok lar... Joking wif u oni...,6,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,155
3,ham,U dun say so early hor... U c already then say...,11,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,61


In [7]:
max(df["message_len"])

910

In [8]:
df.describe()

,words_count,message_len
count,5572.000000,5572.000000
mean,15.606784,80.118808
std,11.422465,59.690841
min,1.000000,2.000000
25%,7.000000,36.000000
50%,12.000000,61.000000
75%,23.000000,121.000000
max,171.000000,910.000000


In [9]:
balance_counts = df.groupby('target')['target'].agg('count').values
balance_counts

array([4825,  747])

In [10]:
ham_df = df[df['target']=="ham"]["message_len"].value_counts().sort_index()
spam_df = df[df['target']=="spam"]["message_len"].value_counts().sort_index()

In [11]:
print(ham_df.count())
print(ham_df.max())

272
123


In [12]:
print(spam_df.count())
print(spam_df.max())

122
39


In [13]:
concat = pd.concat([ham_df,spam_df], axis=1)
concat

,count,count
message_len,,
2,4.0,NaN
3,9.0,NaN
4,6.0,NaN
5,11.0,NaN
6,3.0,NaN
...,...,...
632,1.0,NaN
790,1.0,NaN
910,1.0,NaN


In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = ham_df.index,
    y = ham_df.values,
    name='ham',
    fill='tozeroy',
    marker_color = primary_blue
))
fig.add_trace(go.Scatter(
    x=spam_df.index,
    y=spam_df.values,
    name='spam',
    fill='tozeroy',
    marker_color=primary_grey,
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Data Roles in Different Fields</span>'
)
fig.update_xaxes(range=[0, 70])
fig.show()

# Data Pre-Processing

## Cleaning

In [15]:
def clean_text(text):

    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure','c']
    text = str(text).lower()
    text = re.sub('\[.*?\]','', text)
    text = re.sub('https?://\S+|www\.\S+', '',text)
    text = re.sub('<.*?>+','',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n','',text)
    text = re.sub('\w*\d\w*','',text)
    text = ' '.join(word for word in text.split(' ') if word not in STOPWORDS)
    return text

In [16]:
df['clean_message'] = df['message'].apply(clean_text)
df['clean_message_words_count'] = df['clean_message'].apply(lambda x:len(x.split(' ')))
df['clean_message_len'] = df['clean_message'].apply(len) 
df.head()

,target,message,words_count,message_len,clean_message,clean_message_words_count,clean_message_len
0,ham,"Go until jurong point, crazy.. Available only ...",20,111,go jurong point crazy available bugis n great ...,16,82
1,ham,Ok lar... Joking wif u oni...,6,29,ok lar joking wif oni,5,21
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,155,free entry wkly comp win fa cup final tkts m...,23,103
3,ham,U dun say so early hor... U c already then say...,11,49,dun say early hor already say,6,29
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,61,nah think goes usf lives around though,7,38
